In [ ]:
import sys
sys.path.append('../')
sys.path.append('../tests/python-support/')

%matplotlib notebook

In [ ]:
import matplotlib
import experiments
import itertools
import util
import numpy as np

from pprint import pprint

In [ ]:
# Names of experiments to run
experiment_names = [ "inhomogeneous_dirichlet_unit_square" ]

# Spatial properties. Use integer powers so that we can have reliable filenames
powers = np.array([2, 3, 4 ])
basis_filenames = [ "basis_{}.h5".format(i) for i in powers]
h = 0.5 ** powers
oversampling = [ 2 ]
integrator = [ "direct_crank_nicolson", "iterative_crank_nicolson" ]

# Temporal details of the simulation
end_time = [0.5]
sample_count = [ 401 ]

# Whether to export or import basis
export_basis = False
import_basis = True

def make_basis_file_corrections(params):
    file = params.pop('basis_file')
    
    if export_basis:
        params['basis_export_file'] = file
    if import_basis:
        params['basis_import_file'] = file
    return params

param_collection = itertools.product(end_time, sample_count, oversampling, zip(h, basis_filenames), integrator)
params = [{ 
            'end_time': x[0], 
            'sample_count': x[1], 
            'oversampling': x[2],
            'mesh_resolution': x[3][0],
            'basis_file': x[3][1],
            'integrator': x[4]
          }
          for x in param_collection]

params = map(make_basis_file_corrections, params)
experiment_input = [ { 'experiment': e, 'parameters': p }
                     for e, p in itertools.product(experiment_names, params) ]

In [ ]:
# Run experiments and collect results
results = experiments.run_experiments(experiment_input)

In [ ]:
#pprint(results)

In [ ]:
def key(experiment):
    params = experiment['parameters']
    name = experiment['experiment']
    return (name, params['integrator_name'], params['oversampling'])

results = sorted(results, key = key)
grouped_results = itertools.groupby(results, key = key)
for (name, integrator, oversampling), result in grouped_results:
    result = list(result)
    num_dof = [ result['mesh']['num_vertices'] for result in result ]
    l2_error = [ result['error_summary']['l2'] for result in result ]
    h1_error = [ result['error_summary']['h1'] for result in result ]
    h1_semi_error = [ result['error_summary']['h1_semi'] for result in result ]
    l2_slope = util.estimate_slope(num_dof, l2_error)
    h1_slope = util.estimate_slope(num_dof, h1_error)
    
    print("Experiment name: {name}".format(name=name))
    print("Integrator:      {}".format(integrator))
    print("Oversampling:    {}".format(oversampling))
    print("Dof:      {}".format(num_dof))
    print("H1 semi:  {}".format(h1_semi_error))
    print("H1:       {}".format(h1_error))
    print("L2:       {}".format(l2_error))
    print("H1 slope: {}".format(h1_slope))
    print("L2 slope: {}".format(l2_slope))
    print("")
    
    